# MLE

## Thumbtack Question
* A gambling site with a game of filipping a thumbtack

In [136]:
import sys
import numpy as np
from IPython.display import clear_output

class Thumbtack(object):
    def __init__(self, theta=0.4, start_money=100, rate=2):
        """
        nails up case = H
        nails down case = T
        """
        self.theta = theta # P(H)
        self.money = start_money
        self.rate = rate
        self.nhops = 1
        
        self.bet_switch = False
        self.x = None
        
    def start(self):
        if self.check_left_money():
            clear_output()
            self.bet_switch = True
            print("[Money Left] $ {}".format(self.money))
            self.bet()
            print("Running...")
            result = self.simulate_binomial()
            self.print_result(result)
        else:
            self.recharge()
        
            
    def simulate_binomial(self):
        toss = np.random.binomial(self.nhops, self.theta)
        return toss
    
    def print_result(self, result):
        if result:
            result = "Win"
            self.money += self.bet_amount * self.rate
        else:
            result = "Lose"
            self.money -= self.bet_amount * self.rate
            
        print(result + "!!!")
        print("="*30)
        print("[Money Left] $ {}".format(self.money))
        
    def bet(self):
        while self.bet_switch:
            self.x = input("your bet: ")
            try:
                # check type
                isinstance(int(self.x), int)
                self.x = int(self.x)
                if self.check_money():
                    self.bet_switch = False
            except ValueError:
                print("betting number must be int type. ex) 10, 4, 2 ...")
                
    def check_money(self):
        if self.money - self.x < 0:
            print("You don't have enough money. [Money Left] $ {}".format(self.money) )
            return False
        elif self.x == 0:
            print("bet must be larger than 0")
            return False
        else:
            self.bet_amount = self.x
            self.x = None
            return True
        
    def check_left_money(self):
        if self.money > 0:
            return True
        else:
            return False
    
    def recharge(self):
        if self.check_left_money():
            print("Cannot recharge, Since You have money. [Money Left] $ {}".format(self.money))
            return None 
        else:
            print("You don't have enough money. Get you money!!")
            recharge_switch = True
            while recharge_switch:
                
                re_money = input("rechange amount: ")
                try:
                    # check type
                    isinstance(int(re_money), int)
                    re_money = int(re_money)
                    if re_money > 100:
                        print("cannot recharge money larger than $100.")
                    else:
                        self.money = re_money
                        recharge_switch = False
                except ValueError:
                    print("money must be int type. ex) 10, 4, 2 ...")

In [137]:
game = Thumbtack()

In [ ]:
game.start()

## Binomial Distribution
A discrete probability distritubution:
* Of the number of successes in a sequence of n independent yes/no experiments, and each success has the probability of $\theta$ - Bernoulli experiment
* Flips Condition: i.i.d
    * Independent events
    * Identically distributed according to binomial distribution

So, Let's say nails up case as "H", nails down case as "T". Then we can define probability

$\begin{cases} P(H) = \theta \\ P(T) = 1 - \theta \end{cases}$



If there is a simulation data "HHTHT", then we can calculate its probability:

$$P(HHTHT) = \theta \theta (1- \theta) \theta (1- \theta) = \theta^3 (1-\theta)^2$$

Let's say:
* $D$ as Data = "H,H,T,H,T"
* $n = 5$
* $k = a_H=3$
* $p=\theta$

The probability of simulation data "D" given $\theta$ can be define as below:

$$P(D \vert \theta) = \theta^{a_H}(1-\theta)^{a_T}$$

## Maximum Likelihood Estimation

* Data: we have observed the sequence data of $D$ with $a_H$ and $a_T$ 
* hypothesis: the gambling result of thumbtack follows the binomial distribution of $\theta$
* How to make our hypothesis strong? -> find the model($\theta$) that can well explain the data
    * Find out a better distribution of the observation
    * Find out the best candidate of $\theta$

**MLE(Maximum Likelihood Estimation)**
* Choose $\theta$ that maximize the probability of observed data

$$\hat{\theta} = \underset{\theta}{\arg \max} P(D \vert \theta))$$

## Application to Thumbtack: MLE Calculation

$$\hat{\theta} = \underset{\theta}{\arg \max}\ P(D \vert \theta)) = \underset{\theta}{\arg \max}\ \theta^{a_H}(1-\theta)^{a_T} \quad \cdots (1)$$

However, it is hard to calculate, so we use log function technic.
* Since log function is monotonic increase function, so 

$$\underset{\theta}{\arg \max}\ P(D \vert \theta) = \underset{\theta}{\arg \max}\ \ln \big( P(D \vert \theta) \big)$$

Then our function $(1)$ can be written like this,

$$\begin{aligned} 
\hat{\theta} &= \underset{\theta}{\arg \max}\ \ln \big( P(D \vert \theta) \big) \\
&= \underset{\theta}{\arg \max}\ \ln \big( \theta^{a_H}(1-\theta)^{a_T} \big) \\
&= \underset{\theta}{\arg \max}\ a_H \ln \theta + a_T \ln (1-\theta) \quad \cdots (2)
\end{aligned}$$

Then, we do optimization using derivative setting $(2)$ to zero.

$$\begin{aligned} 
\dfrac{d}{d \theta} \big( a_H \ln \theta + a_T \ln (1-\theta) \big) = 0 \\
\dfrac{a_H}{\theta} - \dfrac{a_T}{\theta} = 0\\
\end{aligned}$$

So, $\theta = \dfrac{a_H}{a_H+a_T}$, and this $\theta$ becomes the best candidate from the MLE perspective.

$$\hat{\theta} = \dfrac{a_H}{a_H+a_T}$$

## Number of Trials

Remember $\hat{\theta}$ is just an inference(guess) from our data, it is not the "real" parameter in the world.

* Additional trials reduce the error of our estimation

Let's say $\theta^*$ is the true parameter of the thumbtack flipping for any error, $\varepsilon > 0$

simple upper bound on the probability provided by [Hoeffding's inequality](https://en.wikipedia.org/wiki/Hoeffding%27s_inequality)

$$P(\vert \hat{\theta} - \theta^* \vert \geq \varepsilon) \leq 2e^{-N\varepsilon^2}$$

* $N$: Number of Trials
* $\varepsilon$: error bound 
* when $\varepsilon$ or $N$ get bigger, the probability of error bound will be smaller.

This process is **Probably Approximate Correct (PAC)** learning. [Wiki link](https://en.wikipedia.org/wiki/Probably_approximately_correct_learning)